<a href="https://colab.research.google.com/github/NoraGraves/AIAttackDemo/blob/main/FacialRecognitionDemonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

We need to import all of these code libraries in order to format our own photos, and to use the pretrained neural network.

In [ ]:
%%capture
%pip install opencv-python
%pip install progressbar2
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 "shape_predictor_68_face_landmarks.dat.bz2"

Now we will clone the repository in Github containing all of the code and sample images we will need.

In [ ]:
%%capture
!git clone 'https://github.com/NoraGraves/AIAttackDemo.git'

In [ ]:
import AIAttackDemo.image_processing as image_processing
import AIAttackDemo.neural_net as neural_net
import AIAttackDemo.attack as attack

from google.colab.patches import cv2_imshow as show_image

# Upload and crop an image of your face!

The first step is to upload an image of your face to this Colab notebook. If you don't feel comfortable using your own face, any picture with a face in it will work.

**Make sure there is only one face in the image!**

In [ ]:
from google.colab import files
uploaded=files.upload()

# saves filepath for uploaded image
filepath = ''
for key in uploaded.keys():
    filepath = f'/content/{key}'

In [ ]:
# Check that the image uploaded correctly by displaying it here!
show_image(image_processing.read_image_from_file(filepath))

Our neural network was trained on pictures of faces that were all in the same format: a square 224x224 image with the face centered inside. Therefore, in order to use your own face as input, you must make sure your picture is in that same format.

The following code will detect the face in the image and crop the image around it.

In [ ]:
# Here, we crop the image and then display it
crop_image = image_processing.crop_image_from_file(filepath)
show_image(crop_image)

# Who do you look like?

In [ ]:
# Upload the model
model = neural_net.load_model_from_file('/content/AIAttackDemo/celeb_faces.model.keras')

It's finally time to run the program, and find out which celebrity you look like!

NOTE: This is a very simple neural network, so the results will not always be perfect. For example, it may say you look like a celebrity of a different gender. Please don't be offended!

In [ ]:
# Turn the image into an input the model can understand
model_input = image_processing.preprocess_image(crop_image)
prediction = neural_net.predict(model_input, model)
show_image(image_processing.read_image_from_file(neural_net.index2filepath(prediction)))

# Time to attack the model!

We will now add noise to the image, so that the model identifies the same person as a different celebrity.

Pick a celebrity and run the code block. This will display an image of the selected celebrity.

In [ ]:
# @title  { run: "auto", display-mode: "form" }
target_name = "Sandra Oh" # @param ['Alica Schmidt', 'Angela Merkel', 'Barack Obama','Bruno Mars', 'Dwayne Johnson', 'Ed Sheeran', 'Emma Stone', 'Greta Thunberg', 'Jackie Chan','Malala', 'Manuel Neuer', 'Mark Forster','Michael Jordan', 'Namika', 'Olaf Schulz','Olivia Rodrigo', 'Rihanna', 'Ryan Gosling','Sandra Oh', 'Serena Williams', 'Simu Lui', 'Zendaya'] {run: "auto"}

sel_index = neural_net.name2index(target_name)
show_image(image_processing.read_image_from_file(neural_net.index2filepath(sel_index)))



Run the attack!

NOTE: This typically takes around 15 minutes

In [ ]:
attack_image = attack.targeted_attack(model, crop_image, target_name)

Let's look at the image, and then see what the model predicts!

In [ ]:
print('Attack Image:')
show_image(attack_image)
print('Original Image:')
show_image(crop_image)

In [ ]:
attack_input = image_processing.preprocess_image(attack_image)
prediction = neural_net.predict(attack_input, model)
show_image(image_processing.read_image_from_file(neural_net.index2filepath(prediction)))

# Further Exploration
Here are some more ideas of things to try if you'd like to keep exploring attacks on neural networks! You can also go back to the beginning of this notebook and try again with a different original picture.

## Attack Strength

Right now, the attack is set to run for up to 400 steps (which takes about 15 minutes). However, this is not guaranteed to reach the target, which might take more time. If your attack did not lead to the celebrity prediction you wanted, try again with more steps!

Running this code block will run the attack for up to the given number of time steps, then show the images and the final prediction.

NOTE: the attack will always end early once it succeeds.

In [ ]:
# @title  { display-mode: "form" }
target_name = 'Angela Merkel' # @param ['Alica Schmidt', 'Angela Merkel', 'Barack Obama','Bruno Mars', 'Dwayne Johnson', 'Ed Sheeran', 'Emma Stone', 'Greta Thunberg', 'Jackie Chan','Malala', 'Manuel Neuer', 'Mark Forster','Michael Jordan', 'Namika', 'Olaf Schulz','Olivia Rodrigo', 'Rihanna', 'Ryan Gosling','Sandra Oh', 'Serena Williams', 'Simu Lui', 'Zendaya']
attack_strength = 'weak: 200 steps, ~6 minutes' # @param ['weak: 200 steps, ~6 minutes', 'medium: 400 steps, ~15 minutes', 'strong: 800 steps, ~30 minutes']
if attack_strength == 'weak: 200 steps, ~6 minutes': steps = 200
elif attack_strength == 'medium: 400 steps, ~15 minutes': steps = 400
else: steps = 800

# Run the attack
attack_image = attack.targeted_attack(model, crop_image, target_name, steps=steps)

# Show the original and attacked images
print('Attack Image:')
show_image(attack_image)
print('Original Image:')
show_image(crop_image)

# Predict
attack_input = image_processing.preprocess_image(attack_image)
prediction = neural_net.predict(attack_input, model)
show_image(image_processing.read_image_from_file(neural_net.index2filepath(prediction)))

## More Detailed Predictions (with Probabilities)

If you want, you can see the probability the model assigns for all of the celebrity options before and after the attack.

In [ ]:
print('Probabilities before the attack:')
neural_net.print_all_probs(model_input, model)

In [ ]:
print('Probabilities after the most recent attack:')
neural_net.print_all_probs(attack_input, model)

## Untargeted Attack

What if you don't care what celebrity the model assigns, as long as it's different from the original output?

Use an untargeted attack!

This will be much faster (~1 minute), and the image will look much more similar to the original.

In [ ]:
# Run the attack
attack_image = attack.untargeted_attack(model, crop_image, steps=200)

# Show the original and attacked images
print('Attack Image:')
show_image(attack_image)
print('Original Image:')
show_image(crop_image)

# Predict
attack_input = image_processing.preprocess_image(attack_image)
prediction = neural_net.predict(attack_input, model)
show_image(image_processing.read_image_from_file(neural_net.index2filepath(prediction)))